In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip chest-xray-pneumonia.zip &>/dev/null
!pip install datasets &>/dev/null
!pip install transformers &>/dev/null
!pip install fastai &>/dev/null
!pip install wandb &>/dev/null

100% 2.29G/2.29G [00:11<00:00, 226MB/s]
100% 2.29G/2.29G [00:11<00:00, 220MB/s]


In [ ]:
labels = ['PNEUMONIA', 'NORMAL']
img_size = 224
def get_training_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from datasets import DatasetDict, Dataset
from transformers import ViTFeatureExtractor, ViTImageProcessor
from datasets import load_metric
from torchvision.transforms import Compose, Normalize, RandomPerspective, ColorJitter, RandomHorizontalFlip
import wandb; wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#train = get_training_data('chest_xray/chest_xray/train')
test = get_training_data('chest_xray/chest_xray/test')
val = get_training_data('chest_xray/chest_xray/val')

OpenCV(4.7.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.7.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



<ipython-input-4-59b9d7d64a00>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


In [ ]:
!cp /content/drive/MyDrive/Projects/train.pt .
train = torch.load('/content/drive/MyDrive/Projects/train.pt')
train_labels = [int(_[1].item()) for _ in train]
train = torch.load('/content/drive/MyDrive/Projects/train.pt')
train_px_vals = [_[0] for _ in train]

In [ ]:
def gray2color(gray_image):
  gray_image = gray_image.unsqueeze(-1)
  return torch.cat([gray_image, gray_image, gray_image], dim = -1)
  #return gray_image.repeat(1, 1, 3)

In [ ]:
train_data = {
    'pixel_values': train_px_vals,
    'label': train_labels
}
val_data = {
    'pixel_values': [_ for _ in val[:, 0]],
    'label': [_ for _ in val[:, 1]]
}

In [ ]:
train_dset = Dataset.from_dict(train_data)
val_dset = Dataset.from_dict(val_data)

In [ ]:
del train_labels
del train_px_vals
del train_data
del val_data
del train

In [ ]:
image_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224', do_resize = True)
image_processor_2 = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224', do_resize = False)

#feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224', do_resize = True)

In [ ]:
im = gray2color(torch.tensor(train_dset[9]['pixel_values']))
#im_feat = feature_extractor(im, return_tensors = 'pt')['pixel_values'][0]
im_proc = image_processor(im, return_tensors = 'pt')['pixel_values'][0]
im_proc2 = image_processor_2(im, return_tensors = 'pt')['pixel_values'][0]

In [ ]:
def transforms(example):
  processed_example = torch.tensor(example['pixel_values'])
  processed_example = gray2color(processed_example)
  processed_example = image_processor(processed_example, return_tensors = 'pt')
  return {
      'pixel_values': processed_example['pixel_values'],
      'label': torch.tensor(example['label'])
  }
def test_transforms(example):
  processed_example = torch.tensor(example['pixel_values']) /255.
  processed_example = gray2color(processed_example)
  processed_example = image_processor(processed_example, return_tensors = 'pt')
  return {
      'pixel_values': processed_example['pixel_values'],
      'label': torch.tensor(example['label'])
  }

In [ ]:
train_dset = train_dset.with_transform(transforms)
val_dset = val_dset.with_transform(transforms)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'

In [ ]:
def collate_fn(batch):
  return {
      'pixel_values': torch.stack([pixel_vals['pixel_values'] for pixel_vals in batch]),
      'labels': torch.stack([pixel_vals['label'] for pixel_vals in batch])
  }

In [ ]:
metric = load_metric('accuracy')
def compute_metrics(p):
  return metric.compute(
      predictions = np.argmax(p.predictions, axis = 1),
      references = p.label_ids
  )

<ipython-input-18-60bcc6e56918>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = './pneumonia_classifier',
    per_device_train_batch_size = 16,
    evaluation_strategy = 'steps',
    num_train_epochs = 2,
    save_steps = 200,
    eval_steps = 200,
    logging_steps = 20,
    learning_rate = 2e-4,
    save_total_limit = 2,
    remove_unused_columns = False,
    push_to_hub = False,
    load_best_model_at_end = True,
    report_to="wandb"
)

In [ ]:
from transformers import ViTForImageClassification
labels = ['PNEUMONIA', 'NORMAL']
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    num_labels = len(labels),
    ignore_mismatched_sizes = True
)
model.to(device);

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = collate_fn,
    compute_metrics = compute_metrics,
    train_dataset = train_dset,
    eval_dataset = val_dset,
    tokenizer = image_processor
)

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics('train', train_results.metrics)
trainer.save_metrics('train', train_results.metrics)
trainer.save_state()
wandb.finish()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: deedax. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy
200,0.084700,0.539176,0.687500
400,0.098100,0.521198,0.937500
600,0.045300,0.008659,1.000000
800,0.001500,0.001734,1.000000


***** train metrics *****
  epoch                    =          2.0
  total_flos               = 1099294038GF
  train_loss               =       0.0695
  train_runtime            =   0:18:24.97
  train_samples_per_second =       13.785
  train_steps_per_second   =        0.862


eval/accuracy,▁▇██
eval/loss,██▁▁
eval/runtime,▄▁█▁
eval/samples_per_second,▄█▁▇
eval/steps_per_second,▄█▁▇
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▄▃▅▃▃▆▂▂▃▃▁▁▂▁▁▁▂▂▁▁▁▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁


In [ ]:
test_data = {
    'pixel_values': [_ for _ in test[:, 0]],
    'label': [_ for _ in test[:, 1]]
}
test_dset = Dataset.from_dict(test_data)
test_dset = test_dset.with_transform(test_transforms)

In [ ]:
wandb.init()
metrics = trainer.evaluate(test_dset)
trainer.log_metrics('eval', metrics)
trainer.save_metrics('eval', metrics)
wandb.finish()

***** eval metrics *****
  epoch                   =        2.0
  eval_accuracy           =     0.8365
  eval_loss               =      0.937
  eval_runtime            = 0:00:29.12
  eval_samples_per_second =     21.423
  eval_steps_per_second   =      2.678


eval/accuracy,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁
train/global_step,▁
eval/accuracy,0.83654
eval/loss,0.93699
eval/runtime,29.1272
eval/samples_per_second,21.423


In [ ]:
import random
idx = random.choice(range(624))
with torch.no_grad():
  logits = trainer.model(test_dset[idx]['pixel_values'][None, ...].to(device)).logits
"Index:", idx, "Predicted:", logits.argmax().item(), "Actual:", test_dset[idx]['label'].item()

('Index:', 558, 'Predicted:', 0, 'Actual:', 1)